In [2]:
import os

curr_dir = os.path.split(os.getcwd())[1]
if curr_dir != "irl-environment-design":
    os.chdir("..")

import numpy as np

from src.utils.make_environment import (
    transition_matrix,
    Environment,
    insert_walls_into_T,
)

from src.utils.constants import ParamTuple

np.set_printoptions(linewidth=160, precision=2)

# from src.utils.environment_design import EnvironmentDesign

## Make true environment

In [3]:
# 0. Setup
## 0.2 Setup the environment
N, M = 7,7

# TODO describe world
agent_R = np.zeros((N, M))
agent_R[N-1, 0] = 1
agent_R[N-1, M-1] = 3



# Start by making the agent we want to learn the policy of
agent_p = 0.7
agent_gamma = 0.6
p_true=0.7

agent_R = agent_R.flatten()
goal_states = np.where(agent_R != 0)[0]

true_params = ParamTuple(agent_p, agent_gamma, agent_R)

wall_states = [14] #TODO: why do we need this

T_true = transition_matrix(N, M, p=p_true, absorbing_states=goal_states)
T_True = insert_walls_into_T(T=T_true, wall_indices=wall_states)

big_small = Environment(
    N=N,
    M=M,
    T_true=T_true,
    wall_states=wall_states,
    R_true=agent_R,
    start_state=0,
    goal_states=goal_states
)

In [4]:
'''
user parameters consist of
- parametrizations of reward, transition and gamma that user supplies
- ranges for the unknown parameters
- resolution of the grid search
'''

'\nuser parameters consist of\n- parametrizations of reward, transition and gamma that user supplies\n- ranges for the unknown parameters\n- resolution of the grid search\n'

## Make custom parametrizations

In [6]:
def custom_big_small_reward_func(N, M, reward_ratio: float):

    '''
    Generates a big-small like reward function.

    Args:
    - N: int, number of rows in the grid
    - M: int, number of columns in the grid
    - reward_ratio: float, ratio of the reward in the bottom right corner to the reward at the bottom left corner
    '''

    _R = np.zeros((N, M))
    _R[N-1, 0] = 1
    _R[N-1, M-1] = reward_ratio

def custom_T(N, M, p: float, goal_states, wall_states):

    _T = transition_matrix(N, M, p=p, absorbing_states=goal_states)
    _T = insert_walls_into_T(T=_T, wall_indices=wall_states)

    return _T

resolution = 15
range_rewards = np.linspace(start = 1, stop = 5, num = resolution)
range_p = np.linspace(start = 0.5, stop = 0.95, num = resolution)
range_gamma = np.linspace(start = 0.5, stop = 0.95, num = resolution)

paramter_ranges = [range_p, range_gamma, range_rewards]

In [7]:
np.zeros(3*[resolution]).shape

(15, 15, 15)

In [8]:
len(paramter_ranges)

3

In [9]:
import itertools

for idx, param in enumerate(itertools.product(*paramter_ranges)):
    print(param)
    print(idx)

(0.5, 0.5, 1.0)
0
(0.5, 0.5, 1.2857142857142856)
1
(0.5, 0.5, 1.5714285714285714)
2
(0.5, 0.5, 1.8571428571428572)
3
(0.5, 0.5, 2.142857142857143)
4
(0.5, 0.5, 2.4285714285714284)
5
(0.5, 0.5, 2.7142857142857144)
6
(0.5, 0.5, 3.0)
7
(0.5, 0.5, 3.2857142857142856)
8
(0.5, 0.5, 3.571428571428571)
9
(0.5, 0.5, 3.8571428571428568)
10
(0.5, 0.5, 4.142857142857142)
11
(0.5, 0.5, 4.428571428571429)
12
(0.5, 0.5, 4.7142857142857135)
13
(0.5, 0.5, 5.0)
14
(0.5, 0.5321428571428571, 1.0)
15
(0.5, 0.5321428571428571, 1.2857142857142856)
16
(0.5, 0.5321428571428571, 1.5714285714285714)
17
(0.5, 0.5321428571428571, 1.8571428571428572)
18
(0.5, 0.5321428571428571, 2.142857142857143)
19
(0.5, 0.5321428571428571, 2.4285714285714284)
20
(0.5, 0.5321428571428571, 2.7142857142857144)
21
(0.5, 0.5321428571428571, 3.0)
22
(0.5, 0.5321428571428571, 3.2857142857142856)
23
(0.5, 0.5321428571428571, 3.571428571428571)
24
(0.5, 0.5321428571428571, 3.8571428571428568)
25
(0.5, 0.5321428571428571, 4.14285714285714

In [11]:
np.zeros(3*[10]).shape

(10, 10, 10)

In [ ]:
env_design = EnvironmentDesign(base_environment=big_small, user_params=true_params, learn_what = "gamma, T")